In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import re
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import joblib

In [1]:
import sklearn
print(sklearn.__version__)


1.5.1


In [9]:
skills_df = pd.read_excel('C:\\Users\\athar\\Big-Data-Project\\project_data\\2023-33\\skills.xlsx', sheet_name = 2, header = 1)

In [11]:
skills_df = skills_df.rename(columns = {"2023 National Employment Matrix title" : "Occupation","Typical education needed for entry" : "Education","Median annual wage, dollars, 2023[1]" : "Salary"})

In [13]:
skills_df = skills_df.drop(columns = ['2023 National Employment Matrix code','Employment, 2023','Employment, 2033','Employment change, numeric, 2023–33','Employment change, percent, 2023–33'])

In [15]:
skills_df.head()

,Occupation,Salary,Education,Adaptability,Computers and information technology,Creativity and innovation,Critical and analytical thinking,Customer service,Detail oriented,Fine motor,Interpersonal,Leadership,Mathematics,Mechanical,Physical strength and stamina,Problem solving and decision making,Project management,Science,Speaking and listening,Writing and reading
0,"Total, all occupations",48060,—,3.8,3.0,2.7,3.1,3.3,3.6,2.5,3.5,3.2,2.6,1.9,2.2,3.5,2.7,1.7,3.2,3.3
1,Chief executives,206680,Bachelor's degree,4.2,3.7,3.5,4.1,3.9,3.7,1.7,4.0,4.5,3.3,1.5,1.3,4.6,4.1,1.8,4.0,4.2
2,General and operations managers,101280,Bachelor's degree,4.1,3.5,3.1,3.6,3.6,3.6,1.7,3.7,4.0,3.0,1.7,1.7,4.1,3.6,1.7,3.5,3.8
3,Legislators[2],47290,Bachelor's degree,4.2,3.7,3.5,4.1,3.9,3.7,1.7,4.0,4.5,3.3,1.5,1.3,4.6,4.1,1.8,4.0,4.2
4,Advertising and promotions managers,131870,Bachelor's degree,4.0,3.6,3.5,3.5,3.5,3.5,1.6,3.5,3.2,3.0,1.3,1.3,3.8,3.1,1.4,3.9,3.7


In [17]:
skills_df['Education'].unique()

array(['—', "Bachelor's degree", 'High school diploma or equivalent',
       "Master's degree", "Associate's degree",
       'Postsecondary nondegree award',
       'No formal educational credential', 'Some college, no degree',
       'Doctoral or professional degree', nan], dtype=object)

In [19]:
skills_df['Education'] = skills_df['Education'].fillna('0')

In [21]:
skills_df['Education'] = skills_df['Education'].map({'-' : 0, 'No formal educational credential' : 0,'High school diploma or equivalent' : 1, 'Some college, no degree' : 1, 'Postsecondary nondegree award' : 1,'Associate\'s degree' : 2, 'Bachelor\'s degree' : 3, 'Master\'s degree' : 4, 'Doctoral or professional degree' : 5})

In [23]:
skills_df.head()

,Occupation,Salary,Education,Adaptability,Computers and information technology,Creativity and innovation,Critical and analytical thinking,Customer service,Detail oriented,Fine motor,Interpersonal,Leadership,Mathematics,Mechanical,Physical strength and stamina,Problem solving and decision making,Project management,Science,Speaking and listening,Writing and reading
0,"Total, all occupations",48060,NaN,3.8,3.0,2.7,3.1,3.3,3.6,2.5,3.5,3.2,2.6,1.9,2.2,3.5,2.7,1.7,3.2,3.3
1,Chief executives,206680,3.0,4.2,3.7,3.5,4.1,3.9,3.7,1.7,4.0,4.5,3.3,1.5,1.3,4.6,4.1,1.8,4.0,4.2
2,General and operations managers,101280,3.0,4.1,3.5,3.1,3.6,3.6,3.6,1.7,3.7,4.0,3.0,1.7,1.7,4.1,3.6,1.7,3.5,3.8
3,Legislators[2],47290,3.0,4.2,3.7,3.5,4.1,3.9,3.7,1.7,4.0,4.5,3.3,1.5,1.3,4.6,4.1,1.8,4.0,4.2
4,Advertising and promotions managers,131870,3.0,4.0,3.6,3.5,3.5,3.5,3.5,1.6,3.5,3.2,3.0,1.3,1.3,3.8,3.1,1.4,3.9,3.7


In [25]:
skills_df = skills_df.dropna()

In [27]:
skills_df.head()

,Occupation,Salary,Education,Adaptability,Computers and information technology,Creativity and innovation,Critical and analytical thinking,Customer service,Detail oriented,Fine motor,Interpersonal,Leadership,Mathematics,Mechanical,Physical strength and stamina,Problem solving and decision making,Project management,Science,Speaking and listening,Writing and reading
1,Chief executives,206680,3.0,4.2,3.7,3.5,4.1,3.9,3.7,1.7,4.0,4.5,3.3,1.5,1.3,4.6,4.1,1.8,4.0,4.2
2,General and operations managers,101280,3.0,4.1,3.5,3.1,3.6,3.6,3.6,1.7,3.7,4.0,3.0,1.7,1.7,4.1,3.6,1.7,3.5,3.8
3,Legislators[2],47290,3.0,4.2,3.7,3.5,4.1,3.9,3.7,1.7,4.0,4.5,3.3,1.5,1.3,4.6,4.1,1.8,4.0,4.2
4,Advertising and promotions managers,131870,3.0,4.0,3.6,3.5,3.5,3.5,3.5,1.6,3.5,3.2,3.0,1.3,1.3,3.8,3.1,1.4,3.9,3.7
5,Marketing managers,157620,3.0,4.0,3.6,3.4,3.7,3.5,3.4,1.7,3.5,3.8,3.1,1.1,1.3,4.0,3.3,1.3,3.8,3.7


In [29]:
skills_df.columns

Index(['Occupation', 'Salary', 'Education', 'Adaptability',
       'Computers and information technology', 'Creativity and innovation',
       'Critical and analytical thinking', 'Customer service',
       'Detail oriented', 'Fine motor', 'Interpersonal', 'Leadership',
       'Mathematics', 'Mechanical', 'Physical strength and stamina',
       'Problem solving and decision making', 'Project management', 'Science',
       'Speaking and listening', 'Writing and reading'],
      dtype='object')

In [31]:
skill_names = ['Salary', 'Education', 'Adaptability',
       'Computers and information technology', 'Creativity and innovation',
       'Critical and analytical thinking', 'Customer service',
       'Detail oriented', 'Fine motor', 'Interpersonal', 'Leadership',
       'Mathematics', 'Mechanical', 'Physical strength and stamina',
       'Problem solving and decision making', 'Project management', 'Science',
       'Speaking and listening', 'Writing and reading']

In [33]:
skills_df['Salary'].dtype

dtype('O')

In [35]:
skills_df['Salary'].unique()

array([206680, 101280, 47290, 131870, 157620, 135160, 134760, 119200,
       106470, 102340, 169510, 156100, 116970, 136380, 99200, 136350,
       125040, 83770, 104900, 54290, 103460, 102610, 88460, 165370, 63060,
       82380, 73460, 65360, 110680, 157740, 88670, 62850, 77030, 83960,
       75660, 57570, 133560, 84900, 71950, 75050, 74520, 75670, 74740,
       67650, 45730, 89980, 79400, 98580, 99410, 56920, 64160, 74530,
       64340, 74680, 79590, 79880, 61630, 84940, 79420, 99010, 99580,
       77860, 106090, 84300, 48570, 69990, 58530, 49010, 78310, 103800,
       120360, 145080, 71530, 59240, 129840, 101510, 134700, 95360, 99700,
       132270, 101800, 84960, 98540, 104920, 120000, 116440, 83640,
       104110, 108020, 70620, 93310, 79320, 76210, 68540, 130720, 88750,
       100730, 112100, 95890, 138080, 106950, 100090, 103690, 99380,
       100270, 104100, 99510, 100640, 125460, 135690, 111970, 61820,
       69260, 64060, 59260, 77830, 60700, 72800, 65080, 54000, 62610,
      

In [37]:
#Cleaning the salary column
skills_df['Salary'] = skills_df['Salary'].replace('—', '0')
skills_df['Salary'] = skills_df['Salary'].replace('>=239,200', '239200')
# skills_df['Salary'] = skills_df['Salary'].astype(int)



In [39]:
skills_df = skills_df.dropna()

In [41]:
skills_df['Salary'] = skills_df['Salary'].astype(int)

In [43]:
X = skills_df.drop(['Salary','Occupation'],axis = 1)
y = skills_df['Salary']

In [45]:
y.unique()

array([206680, 101280,  47290, 131870, 157620, 135160, 134760, 119200,
       106470, 102340, 169510, 156100, 116970, 136380,  99200, 136350,
       125040,  83770, 104900,  54290, 103460, 102610,  88460, 165370,
        63060,  82380,  73460,  65360, 110680, 157740,  88670,  62850,
        77030,  83960,  75660,  57570, 133560,  84900,  71950,  75050,
        74520,  75670,  74740,  67650,  45730,  89980,  79400,  98580,
        99410,  56920,  64160,  74530,  64340,  74680,  79590,  79880,
        61630,  84940,  79420,  99010,  99580,  77860, 106090,  84300,
        48570,  69990,  58530,  49010,  78310, 103800, 120360, 145080,
        71530,  59240, 129840, 101510, 134700,  95360,  99700, 132270,
       101800,  84960,  98540, 104920, 120000, 116440,  83640, 104110,
       108020,  70620,  93310,  79320,  76210,  68540, 130720,  88750,
       100730, 112100,  95890, 138080, 106950, 100090, 103690,  99380,
       100270, 104100,  99510, 100640, 125460, 135690, 111970,  61820,
      

In [47]:
skills_df.shape

(832, 20)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state= 1)

In [51]:
y_test

74     71530
551    50510
118    77830
373    57930
217    93810
       ...  
812    39110
136    67330
206    85260
312    69680
135    68750
Name: Salary, Length: 167, dtype: int32

In [53]:
model = RandomForestRegressor(n_estimators = 500, random_state = 20)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [55]:
y_pred

array([ 64803.85      ,  90585.06      ,  77441.295     ,  68550.98      ,
        96923.33433333,  79399.091     ,  69167.30333333, 204190.63333333,
       105078.56      ,  81128.02833333,  47658.946     ,  79613.41      ,
        50173.5       ,  46579.35      ,  51595.72      ,  99611.64      ,
        52078.82      ,  47790.32666667,  69415.50333333,  81131.04      ,
        50333.302     ,  69820.35433333,  97671.66      ,  47604.42666667,
        39726.908     ,  47270.14      ,  44122.29333333, 113038.58833333,
        60390.43      ,  42574.78      ,  52234.13      ,  44963.71333333,
        51027.78      ,  43919.448     ,  44353.        ,  38204.7       ,
        45594.4       ,  34844.83833333,  35139.42      ,  45393.8       ,
       108431.48      ,  45456.74      ,  67907.69      ,  62006.9       ,
       104177.70666667, 105079.24033333, 111874.56666667,  46267.24      ,
        69869.35333333,  44859.49333333,  45047.7       ,  43306.33666667,
        56421.08      ,  

In [57]:
mae = mean_absolute_error(y_test, y_pred)
mae

11262.468746481065

In [59]:
mse = mean_squared_error(y_test, y_pred)
mse

287199669.66766375

In [61]:
rmse = mse ** 0.5
rmse

16946.96638539369

In [63]:
r2 = r2_score(y_test, y_pred)
r2

0.7832028503646313

In [65]:
#Predicting Salary of test user.
test_user = [[5,4.5,5,4.1,4.2,4.4,4,4.3,3.2,4.1,5,4.2,3.3,2.1,5,4.4,5,3.9]]
model.predict(test_user)

D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([136288.042])

In [67]:
joblib.dump(model, "rf_salary_model.pkl")

['rf_salary_model.pkl']

In [45]:
param_grid = {
    'n_estimators': [500, 800, 1000],         # Number of trees
    'max_depth': [None, 10, 20, 30],        # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],        # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],          # Minimum samples required to be at a leaf node
    'max_features': ['sqrt', 'log2'] # Number of features to consider when looking for the best split
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='r2')
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=500, random_state=20),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [500, 800, 1000]},
             scoring='r2', verbose=2)

In [46]:
best_model = grid_search.best_estimator_

In [47]:
y_pred = best_model.predict(X_test)
r2_score(y_test, y_pred)

0.7701525443089112